# Agents for Amazon Bedrock - Testing Agent Invocation

This notebook provides sample code for testing the agent invocation using the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime.html) client for `bedrock-agent-runtime`

### Use Case
Now that we've created the agent and attached the knowledge base to it, we will use the runtime client to invoke the agent with different queries. The boto3 sdk for agent is divided into two clients: `bedrock-agent` and `bedrock-agent-runtime`. The `bedrock-agent` client is responsible for the functionalities to create, update, delete and/or prepare an agent or a knowledge base. While the `bedrock-agent-runtime` is responsible for invoke an agent (with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API) and retrieve documents from a knowledge base (with the [`retrieve`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [`retrieve_and_generate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) APIs). This notebook will focus on the runtime invocation of the agent created in the previous notebooks. We will use the `invoke_agent` API.



### Notebook Walkthrough

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Invoke agent with a new session using Knowledge Bases and Action Groups
- Invoke agent with an existent session
- Invoke agent with prompt attributes
- Invoke agent with trace enabled
- Invoke agent with different languages for the request


### Next Steps: 
In the next lab, we will clean up the resources created

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [2]:
%store -r
agent_foundation_model

'amazon.nova-pro-v1:0'

#### Importing required packages and initiate variables

Let's also import the necessary packages, setup the logging and initiate the boto3 client for `bedrock-agent-runtime`. We are also initiating the client for `dynamodb` as a support functionality to check that the tasks were performed correctly by our agent

In [3]:
import boto3
import logging
import pprint
import json
import pandas as pd
from agent import invoke_agent_helper
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
dynamodb = boto3.resource('dynamodb')

#### Create support invokeAgent function

Let's again use the same support function called `invoke_agent_helper` from `agents.py` to allow us to invoke the agent with or without trace enabled and with or without session state. 

This function allows the user to send a `query` to the agent with a `session_id`. The user can then decide to enable trace or not using the `enable_trace` boolean variable and to pass a session state as a dictionary via the `session_state` variable.

If a new `session_id` is provided, the agent will create a new conversation without previous context. If the same `session_id` is reused, the conversation history related to that session id is made available to the agent.

If the `enable_trace` is set to `True`, each response from the agent is accompanied by a *trace* that details the step being orchestrated by the agent. It allows you to follow the agent's (reasoning via Chain of Thoughts prompting) that led to the final response at that point of the conversation.

Finally, you can also pass a session context using the `session_state` parameter. The session state allows you to share the following information with the agent:
- **`sessionAttributes`**: attributes that persist over a session between the user and the agent. All invokeAgent calls with the same session_id belong to the same sesison and will have the sessionAttributes shared with them as long as the session time limit has not being surpassed and the user has not ended the session. The sessionAttributes are available in the lambda function but are **not** added to the agent's prompt. As a result, you can only use session attributes if your lambda function can handle them. You can find more examples of using a session attribute [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/06-prompt-and-session-attributes). It is also a good pattern to implement fine-grained access control for certain APIs using the lambda function integration. You can find an example for it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/09-fine-grained-access-permissions)
- **`promptSessionAttributes`**: attributes that persist over a single invokeAgent call. Prompt attributes are added to the prompt and to the lambda function. You can also use the `$prompt_session_attributes$` placeholder when editing the orchestration base prompt.
- **`invocationId`**: The id returned by the agent in the [ReturnControlPayload](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_ReturnControlPayload.html) object in the returnControl field of the InvokeAgent response. This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).
- **`returnControlInvocationResults`**: the results obtained from invoking the action outside of agents for Amazon Bedrock.  This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).

#### Create support selectAllFromDynamoDB function

We will also create the support function called `selectAllFromDynamoDB` to select all data in the dynamoDB table `restaurant_bookings`. This function will be used to validate our agent's behaviour

In [4]:
def selectAllFromDynamodb():
    # Get the table object
    table = dynamodb.Table(ddb_table_name)

    # Scan the table and get all items
    response = table.scan()
    items = response['Items']

    # Handle pagination if necessary
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [5]:
# test function invocation
items = selectAllFromDynamodb()
items

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,eeec9f71,Anna


### Invoking agent with a new session id
Let's first use the `InvokeAgent` function to query the Knowledge Base with the agent without previous context

In [7]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What is in the childrens menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



The children's menu includes the following items:
- **Entrees:**
  - Chicken Nuggets
  - Macaroni and Cheese
  - Mini Cheese Quesadillas
  - Peanut Butter and Banana Sandwich
  - Veggie Pita Pockets
- **Mains:**
  - Mini Cheeseburgers
  - Fish Sticks




CPU times: user 18.3 ms, sys: 283 μs, total: 18.6 ms
Wall time: 6.5 s


### Invoking agent with existent session id

Next we can use the context to ask a follow up question. To do so, we use the same `session_id` with the `invokeAgent` function

In [8]:
%%time
query = "Which of those options are vegetarian?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



The vegetarian options on the children's menu are:
- Macaroni and Cheese
- Mini Cheese Quesadillas
- Peanut Butter and Banana Sandwich (if using vegetarian peanut butter)
- Veggie Pita Pockets




CPU times: user 14.2 ms, sys: 6.24 ms, total: 20.4 ms
Wall time: 3.49 s


As you can see, the agent knows that we are talking about the children's menu.

### Invoking agent using action group

Now let's use our agent to make a reservation. By doing so, we will require the agent to execute an action from our action group to create a new reservation.

In [9]:
%%time
query = "Hi, I am Maria. I want to create a booking for 4 people, at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



Your booking has been successfully created. Your booking ID is 5ac39b7f. Please keep this ID for any future reference or changes to your booking.



CPU times: user 17.1 ms, sys: 0 ns, total: 17.1 ms
Wall time: 7.46 s


Let's double check that the data was properly added to the dynamoDB table

In [10]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,4,2024-05-05,21:00,5ac39b7f,Maria
1,2,2024-05-05,20:00,eeec9f71,Anna


Great! We've used our agent to do create a reservation. However, often when booking tables in restaurants we are already logged in to a system that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [promptSessionAttributes](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [11]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm on the 6th of May 2024."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)



Your booking has been successfully created. Your booking ID is 6d82b445.



CPU times: user 17.6 ms, sys: 108 μs, total: 17.7 ms
Wall time: 5.96 s


Again let's validate the the correct data was added to dynamoDB

In [12]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-06,20:00,6d82b445,John
1,4,2024-05-05,21:00,5ac39b7f,Maria
2,2,2024-05-05,20:00,eeec9f71,Anna


We can also validate the data using our agent's session information since the agent knows the booking id to invoke the get booking details function

In [14]:
%%time
query = "Get the details for the last booking created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

The details for the last booking created are as follows:
- Booking ID: 6d82b445
- Name: John
- Date: 2024-05-06
- Hour: 20:00
- Number of Guests: 2

CPU times: user 26.1 ms, sys: 0 ns, total: 26.1 ms
Wall time: 6.84 s


#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [16]:
%%time
query = f"I want to delete the booking."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Your booking with ID 6d82b445 has been deleted successfully.
CPU times: user 16.8 ms, sys: 0 ns, total: 16.8 ms
Wall time: 5.29 s


And we confirm that the booking has also been deleted from the dynamoDB table

In [17]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,4,2024-05-05,21:00,5ac39b7f,Maria
1,2,2024-05-05,20:00,eeec9f71,Anna


#### Invoking agent using promptSessionAttributes to handle temporal information

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Agents for Amazon Bedrock also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [22]:
# retrieving today
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
today

'2025-06-02'

In [27]:
%%time
# reserving a table for tomorrow
session_id:str = str(uuid.uuid1())
query = f"Today is {today}. I want to create a booking for 2 people, at 8pm tomorrow."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)



Your booking has been successfully created. The booking ID is cfca59b6.



CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 5.83 s


And now to confirm that everything got added properly, let's retrieve the details of the last booking 

In [29]:
%%time
query = "Could you get the details for the last booking created?"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

The details of your last booking are as follows:
- Booking ID: cfca59b6
- Name: John
- Date: 2025-06-03
- Time: 20:00
- Number of guests: 2

CPU times: user 16.1 ms, sys: 792 μs, total: 16.9 ms
Wall time: 5.26 s


#### Asking the agent for food recomendations

Another good use case for our agent, is to use its reasoning capabilities to ask for some food recommendation. Let's look at a couple of examples for it

In [31]:
%%time
session_id:str = str(uuid.uuid1())
query = "What do you have for kids that don't like fries?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)


We offer several options for kids who don't like fries. These include:
- Chicken Nuggets
- Macaroni and Cheese
- Mini Cheese Quesadillas
- Peanut Butter and Banana Sandwich
- Veggie Pita Pockets
- Mini Cheeseburgers
- Fish Sticks
- Grilled Cheese Sandwich
- Spaghetti with Marinara Sauce
- Mini Pita Pizza




CPU times: user 17.7 ms, sys: 1.22 ms, total: 18.9 ms
Wall time: 7.3 s


In [34]:
%%time
session_id:str = str(uuid.uuid1())
query = "I am allergic to shrimps. What can I eat at this restaurant?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)


Here are some menu items that do not contain shrimp:
- Grilled sirloin steak seasoned with garlic butter, accompanied by loaded mashed potatoes and sautéed vegetables.
- Vegetable stir-fry with tofu and a colorful medley of vegetables in a savory soy-ginger sauce, served over jasmine rice.
- Classic New York cheesecake with a graham cracker crust, topped with a choice of fruit compote or chocolate ganache.
- Warm apple pie with a flaky crust, served with a scoop of vanilla ice cream and a drizzle of caramel sauce.
- Rich and gooey chocolate cake with a molten center, dusted with powdered sugar and served with a scoop of raspberry sorbet.
- Buttery shortbread crust topped with a gooey pecan filling, cut into bars for easy serving.
- Layers of vanilla pudding, sliced bananas, and vanilla wafers, topped with whipped cream and a sprinkle of crushed nuts.
- BBQ Chicken Salad with Ranch.
- Black Bean Veggie Burger with Sweet Potato Fries.
- Chicken nuggets served with a side of ketchup or r

### Invoking agent with trace enabled

We can also invoke our agent with the trace enabled to produce the details of each step being orchestrated by the Agent. Let's see all the steps executed when checking for documents in the knowledge base

In [35]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the desserts on the adult menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=True)
print(response)

[2025-06-02 06:18:36,287] p528 {agent.py:170} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Mon, 02 Jun 2025 06:18:36 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '6ab15c5a-3f79-11f0-8fc3-5e2c208b71ba',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '84f171a0-dbc6-43b8-bb32-aa1ccacef190'},
                      'HTTPStatusCode': 200,
                      'RequestId': '84f171a0-dbc6-43b8-bb32-aa1ccacef190',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f965f64daf0>,
 'contentType': 'application/json',
 'sessionId': '6ab15c5a-3f79-11f0-8fc3-5e2c208b71ba'}


[2025-06-02 06:18:36,536] p528 {agent.py:184} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "NREM92UB3F",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:710299592439:agent-alias/NREM92UB3F/TSTALIASID"
    }
  ],
  "eventTime": "2025-06-02 06:18:36.534908+00:00",
  "sessionId": "6ab15c5a-3f79-11f0-8fc3-5e2c208b71ba",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "foundationModel": "amazon.nova-pro-v1:0",
        "inferenceConfiguration": {
          "maximumLength": 1024,
          "stopSequences": [
            "</answer>",
            "\n\n<thinking>",
            "\n<thinking>",
            " <thinking>"
          ],
          "temperature": 1.0,
          "topK": 1,
          "topP": 1.0
        },
        "text": "{\"system\":\"   Agent Description: You are a restaurant agent, helping clients retrieve information from their booking, create a new booking or delete an existing booking   



The desserts on the adult menu are:
1. Classic New York Cheesecake
2. Apple Pie A La Mode
3. Chocolate Lava Cake
4. Pecan Pie Bars
5. Banana Pudding Parfait




CPU times: user 37.9 ms, sys: 0 ns, total: 37.9 ms
Wall time: 6.29 s


### Invoking agent with different prompt languages

The last feature that we will highlight in this notebook is the ability of LLM models to handle inputs in different languages. We've only implemented our agent in English, but wouldn't it be great if it could also handle other languages?

Good news is that it can! This is due to the LLM multi-language capabilities. And the best part is that we don't need to change anything in the agent and it will even going to respond in the requested language.

Let's try a couple of queries!

In [36]:
%%time
# Invoking agents in spanish
session_id:str = str(uuid.uuid1())
query = "¿Podrías reservar una mesa para dos 25/07/2024 a las 19:30"
session_state = {
    "promptSessionAttributes": {
        "Nombre": "Gabriela"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)


Su reserva ha sido creada con éxito. Su ID de reserva es 942d3e60.


CPU times: user 17.3 ms, sys: 29 μs, total: 17.4 ms
Wall time: 6.39 s


In [37]:
%%time
# Invoking agents in german
session_id:str = str(uuid.uuid1())
query = "Könnten Sie heute Abend einen Tisch für zwei reservieren? Um 19:30 Uhr"
session_state = {
    "promptSessionAttributes": {
        "Name": "Julian",
        "Heute": today
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Könnten Sie bitte die Anzahl der Gäste bestätigen?
CPU times: user 17.3 ms, sys: 952 μs, total: 18.2 ms
Wall time: 2.7 s


Last, let's check our dynamoDB to see all the bookings available

In [38]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2025-06-03,20:00,abf0c343,John
1,2,2025-06-03,20:00,56432ffe,Rui
2,2,2024-07-25,19:30,942d3e60,Gabriela
3,4,2024-05-05,21:00,5ac39b7f,Maria
4,2,2025-06-03,20:00,f4858afe,Rui
5,2,2024-05-05,20:00,eeec9f71,Anna
6,2,2025-06-03,20:00,cfca59b6,John


### Next Steps

Next we will delete all the resources created